In [1]:
import json
import openai 
from dotenv import  load_dotenv, find_dotenv

load_dotenv(find_dotenv())
client = openai.OpenAI()

#### Criando a função que o modelos podera executar

In [2]:
def obter_temperatura_atual(local, unidade="celsius"):
    if "sao paulo" in local.lower():
        return json.dumps({
            "local": "Sao Paulo", 
            "temperatura": 32, 
            "unidade": unidade})
    elif "porto alegre" in local.lower():
        return json.dumps({
            "local": "Porto Alegre",
            "temperatura": 25,
            "unidade": unidade})
    else:
        return json.dumps({
            "local": local,
            "temperatura": "unknown",
})

In [3]:
obter_temperatura_atual("Sao Paulo")

'{"local": "Sao Paulo", "temperatura": 32, "unidade": "celsius"}'

#### Criando um dicionario com a descrição da ferramenta que poderá ser executada

In [4]:
tools = [{
    "type": "function",
    "function": {
        "name": "obter_temperatura_atual",
        "description": "Obtem temperatura atual da cidade",
        "parameters": {
            "type": "object",
            "properties": {
                "local": {
                    "type": "string",
                    "description": "O nome da cidade. Exemplo: São Paulo",
                },
                "unidade": {
                    "type": "string", 
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["local"]
        },
    }
}]

#### Chamando o modelo com a nova ferramenta

In [5]:
mensagens = [{"role":"user", "content": "Qual a temperatura de Porto Alegre agora?"}]

resposta = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = mensagens,
    tools=tools,
    tool_choice="auto"
)

In [6]:
mensagem = resposta.choices[0].message
print(mensagem)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2ovxAXbIWJvYdjzBBgnR1CYf', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')])


In [7]:
print(mensagem.tool_calls)

[ChatCompletionMessageToolCall(id='call_2ovxAXbIWJvYdjzBBgnR1CYf', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]


In [8]:
print(mensagem.content)

None


In [9]:
import json
tool_calls = mensagem.tool_calls[0]
print(tool_calls.function.name)
print(tool_calls.function.arguments)
observacao = obter_temperatura_atual(**json.loads(tool_calls.function.arguments))
observacao

obter_temperatura_atual
{"local":"Porto Alegre","unidade":"celsius"}


'{"local": "Porto Alegre", "temperatura": 25, "unidade": "celsius"}'

#### Chamando novamente o modelo

In [10]:
mensagens.append(mensagem)
mensagens

[{'role': 'user', 'content': 'Qual a temperatura de Porto Alegre agora?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2ovxAXbIWJvYdjzBBgnR1CYf', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')])]

In [11]:
mensagens.append({
    "tool_call_id": "tools_call.id",
    "role": "tool",
    "name": "tools_call.function.name",
    "content": "observacao"
})
mensagens

[{'role': 'user', 'content': 'Qual a temperatura de Porto Alegre agora?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2ovxAXbIWJvYdjzBBgnR1CYf', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]),
 {'tool_call_id': 'tools_call.id',
  'role': 'tool',
  'name': 'tools_call.function.name',
  'content': 'observacao'}]

In [12]:
resposta = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = mensagens,
    tools=tools,
    tool_choice="auto"
)
resposta

BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: 'tool_call_id' of 'tools_call.id' not found in 'tool_calls' of previous message.", 'type': 'invalid_request_error', 'param': 'messages.[2].tool_call_id', 'code': None}}